##  Linear Models: LogisticRegression, SGDClassifier, LinearSVC  

In [1]:
from IPython.display import display, HTML, Markdown
display(HTML("<style>.container { width:90% !important; }</style>"))

import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import joblib
import warnings
warnings.filterwarnings('ignore')

## Separate features and target

In [2]:
# Load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

y_train = train["Score"]
y_test = test["Score"]

X_train = train.drop("Score", axis=1)
X_test = test.drop("Score", axis=1)

## Transformations

In [3]:
# Define features
numeric_features = ["WhiteElo", "EloDif"]
categorical_features = ["Opening_name", "Time_format", "Increment_binary"]

# Preprocessing transformers
numeric_transformer = Pipeline([("scaler", StandardScaler())])
categorical_transformer = Pipeline([("onehot", OneHotEncoder(handle_unknown="ignore"))])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

##  Logistic Regression and GridSearch

In [4]:
lr_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(solver="lbfgs", multi_class="multinomial", max_iter=1000, random_state=42))
])

param_lr = {"classifier__C": [0.01, 0.1, 1, 10]}
grid_search_lr = GridSearchCV(lr_pipeline, param_lr, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_lr.fit(X_train, y_train)

print("Best params (LogReg):", grid_search_lr.best_params_)
best_lr = grid_search_lr.best_estimator_

Best params (LogReg): {'classifier__C': 0.1}


In [5]:
y_pred = best_lr.predict(X_test)
print("Test accuracy: {:.2f}%".format(100 * accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
joblib.dump(best_lr, "best_logistic_regression_model.joblib")

Test accuracy: 53.80%
              precision    recall  f1-score   support

   Black Win       0.53      0.44      0.48      4524
        Draw       0.00      0.00      0.00       566
   White Win       0.54      0.69      0.61      4910

    accuracy                           0.54     10000
   macro avg       0.36      0.38      0.36     10000
weighted avg       0.51      0.54      0.52     10000



['best_logistic_regression_model.joblib']

### The logistic regression baseline is doing a bit better than a completely naïve guesser, but it still struggles badly with the minority “draw” class.

## SGDClassifier and GridSearch

In [6]:
sgd_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", SGDClassifier(random_state=42))
])

param_sgd = {
    "classifier__alpha": [1e-4, 1e-3, 1e-2, 1e-1],              
    "classifier__max_iter": [1000, 2000],
}

grid_sgd = GridSearchCV(sgd_pipeline, param_sgd, cv=3, scoring='accuracy', n_jobs=-1)
grid_sgd.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['WhiteElo',
                                                                          'EloDif']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Opening_name',
                                                                          'Time_format',
                                                                          'Increment_binary'])])),
                                       ('classifier',
                                        SGDClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'classifier__alpha': [0.0001, 0.001, 0.01, 0.1],
                         'classifier__max_iter': [1000, 2000]},
             scoring='accuracy')

In [7]:
print("Best params (SGD):", grid_sgd.best_params_)
best_sgd = grid_sgd.best_estimator_
y_pred = best_sgd.predict(X_test)
print(f"Test accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))
joblib.dump(best_sgd, "best_sgd.joblib")

Best params (SGD): {'classifier__alpha': 0.1, 'classifier__max_iter': 1000}
Test accuracy: 0.54

Classification Report:
               precision    recall  f1-score   support

   Black Win       0.55      0.32      0.40      4524
        Draw       0.00      0.00      0.00       566
   White Win       0.53      0.80      0.64      4910

    accuracy                           0.54     10000
   macro avg       0.36      0.37      0.35     10000
weighted avg       0.51      0.54      0.50     10000



['best_sgd.joblib']

### Our SGDClassifier is doing a bit better than pure chance, but still only modestly above a trivial baseline

## LinearSVC and GridSearch

In [8]:
# LinearSVC
svc_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LinearSVC(random_state=42, dual=False))
])

param_svc = {
    "classifier__C": [0.1, 1, 10],
    "classifier__max_iter": [1000, 2500, 5000, 10000]
}

grid_svc = GridSearchCV(svc_pipeline, param_svc, cv=3, scoring='accuracy', n_jobs=-1)
grid_svc.fit(X_train, y_train)

print("Best params (LinearSVC):", grid_svc.best_params_)
best_svc = grid_svc.best_estimator_
y_pred = best_svc.predict(X_test)
print(f"Test accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Best params (LinearSVC): {'classifier__C': 0.1, 'classifier__max_iter': 1000}
Test accuracy: 0.54

Classification Report:
               precision    recall  f1-score   support

   Black Win       0.53      0.43      0.48      4524
        Draw       0.00      0.00      0.00       566
   White Win       0.54      0.70      0.61      4910

    accuracy                           0.54     10000
   macro avg       0.36      0.38      0.36     10000
weighted avg       0.51      0.54      0.51     10000



In [9]:
joblib.dump(best_svc, "best_linear_svc_model.joblib")

['best_linear_svc_model.joblib']

### LinearSVC has a similar poor performance as our previous models. All models score around 54%.